In [ ]:
#failed_seqs = np.load('/home/msiam/Code/temp/temporal_fss/analysis_npys/sp%s/failed_seqs_betterrepri.npy'%split)
#good_seqs = np.load('/home/msiam/Code/temp/temporal_fss/analysis_npys/sp%s/good_seqs_bettertti.npy'%split)

#print('Failed ', len(failed_seqs), 'Good ', len(good_seqs))
#idx = -2500
#idx = -10
#specific_seq = 'run_0_iter_14579_'
#specific_seq = failed_seqs[idx]#convert_back(sorted(convert(list(failed_seqs)))[idx]) + '_'
#specific_seq = good_seqs[idx]


#main_dir = '/home/msiam/Visualizations/ytvis_split0_failedseqs/'
#main_dir = '/local/data0/ytvis_plots_repaired_global_only/episodes'
#main_dir = '/local/riemann1/home/msiam/tti_minivspw_vis/episodes/'



In [8]:
import os

def parse_dir(main_dir, split, danet=False):
    if danet:
        files_dir = os.path.join(main_dir, 'split_%s'%split, 'preds')
    else:
        files_dir = os.path.join(main_dir, 'tti', 'split_%s'%split, 'preds')
    
    seqs = []
    for file_ in os.listdir(files_dir):
        ftokens = file_.split('.')[0].split('_')[:-1]
        seqname = ''
        for token in ftokens:
            seqname += token + '_'
        
        if seqname not in seqs:
            seqs.append(seqname)
        #print(seqname)
    return seqs


In [16]:
%matplotlib inline
import matplotlib.pyplot as plt
from ipywidgets import interact
from PIL import Image
import os
from sklearn import datasets
import glob
import numpy as np

kshot = 5
 
########################### RePRI + TTI ##################
def browse_images(predsfiles, ttipredsfiles, sprtsfiles, qriesfiles, 
                  specific_seq):
    
    n = len(predsfiles)
    def view_image(i):
        plt.figure(figsize=(30, 10), dpi=80)
        if specific_seq == '':
            fname = convert_back(predsfiles[i])
        else:
            fname = predsfiles[i]
        predimg = Image.open(os.path.join(predsdir, fname))
        plt.subplot(4, 5, 1)
        plt.axis('off')
        plt.imshow(predimg)
        plt.title('File: %s' % predsfiles[i], loc='center')
        
        
        if specific_seq == '':
            fname = convert_back(ttipredsfiles[i])
        else:
            fname = ttipredsfiles[i]
        ttipredimg = Image.open(os.path.join(ttipredsdir, fname))
        plt.subplot(4, 5, 5+1)
        plt.axis('off')
        plt.imshow(ttipredimg)
        
        for j in range(kshot):
            if specific_seq == '':
                fname = convert_back(sprtsfiles[i*kshot+j])
            else:
                fname = sprtsfiles[i*kshot+j]
            sprtimg = Image.open(os.path.join(sprtsdir, fname))
            plt.subplot(4, 5, j+5*2+1)
            plt.axis('off')
            plt.imshow(sprtimg)
        
        if specific_seq == '':
            fname = convert_back(qriesfiles[i])
        else:
            fname = qriesfiles[i]
        qryimg = Image.open(os.path.join(qriesdir, fname))
        plt.subplot(4, 5, 5*3 + 1)
        plt.axis('off')
        plt.imshow(qryimg)
    interact(view_image, i=(0,n-1))

########################### DANet ##################
def browse_images_2(danetpreds, danetqries, danetsprts):
    n = len(danetpreds)
    def view_image(i):
        plt.figure(figsize=(30, 10), dpi=80)
        fname = danetpreds[i]
        predimg = Image.open(fname)
        plt.subplot(4, 5, 1)
        plt.axis('off')
        plt.imshow(predimg)
        plt.title('File: %s' % danetpreds[i], loc='center')
        
        
        fname = danetqries[i]
        ttipredimg = Image.open(fname)
        plt.subplot(4, 5, 5+1)
        plt.axis('off')
        plt.imshow(ttipredimg)
        
        for j in range(kshot):
            fname = danetsprts[i*kshot+j]
            sprtimg = Image.open(fname)
            plt.subplot(8, 5, j+5*2+1)
            plt.axis('off')
            plt.imshow(sprtimg)
        
        plt.show()
    
    interact(view_image, i=(0,n-1))

def convert(array):
    for i in range(len(array)):
        tokens = array[i].split('_')
        indexed_tokens = [1, 3]
        for j in indexed_tokens:
            tokens[j] = '%05d'%int(tokens[j])
        
        new = ''
        for t in tokens:
            new += t + '_'
        array[i] = new[:-1]
    return array

def convert_back(fname):
    tokens = fname.split('_')
    indexed_tokens = [1, 3]
    for j in indexed_tokens:
        tokens[j] = '%d'%int(tokens[j])

    new = ''
    for t in tokens:
        new += t + '_'
    return new[:-1]

main_dir = '/local/riemann1/home/msiam/debug_bdry/ours_vis/episodes/'
split = 0

if not os.path.exists('all_seqs_ytvis_ours.npy'):
    all_seqs = parse_dir(main_dir, '%d'%split)
else:
    all_seqs = np.load('all_seqs_ytvis_ours.npy', allow_pickle=True)

idx = 30
specific_seq = all_seqs[idx]
#print(specific_seq)
predsdir = os.path.join(main_dir, 'repri', 'split_%d'%split, 'preds')
seqs = glob.glob(os.path.join(predsdir, '%s*'%specific_seq))
predsfiles = sorted(seqs)

ttipredsdir = os.path.join(main_dir, 'tti', 'split_%s'%split, 'preds')
seqs = glob.glob(os.path.join(ttipredsdir, '%s*'%specific_seq))
ttipredsfiles = sorted(seqs)

qriesdir = os.path.join(main_dir, 'repri', 'split_%s'%split, 'queries')
seqs = glob.glob(os.path.join(qriesdir, '%s*'%specific_seq))
qriesfiles = sorted(seqs)

sprtsdir = os.path.join(main_dir, 'repri', 'split_%s'%split, 'supports')
seqs = glob.glob(os.path.join(sprtsdir, '%s*'%specific_seq))
sprtsfiles = sorted(seqs)

################ DANet Stuff
if not os.path.exists('all_seqs_ytvis_danet.npy'):
    all_seqs = parse_dir(main_dir.replace('ours_vis/episodes', 'danet_vis'), '%d'%(split+1), True)
else:
    all_seqs = np.load('all_seqs_ytvis_danet.npy', allow_pickle=True)

specific_seq = all_seqs[idx]
danetpredsdir = os.path.join(main_dir.replace('ours_vis/episodes', 'danet_vis'), 'split_%d'%(split+1), 'preds')
danetpredsfiles = sorted(glob.glob(os.path.join(danetpredsdir, '%s*'%specific_seq)))

qriesdir = os.path.join(main_dir.replace('ours_vis/episodes', 'danet_vis'), 'split_%d'%(split+1), 'queries')
danetqriesfiles = sorted(glob.glob(os.path.join(qriesdir, '%s*'%specific_seq)))

sprtsdir = os.path.join(main_dir.replace('ours_vis/episodes', 'danet_vis'), 'split_%d'%(split+1), 'supports')
danetsprtsfiles = sorted(glob.glob(os.path.join(sprtsdir, '%s*'%specific_seq)))

print('Before Calling Browse Images')
browse_images(predsfiles, ttipredsfiles, sprtsfiles, qriesfiles, specific_seq)
browse_images_2(danetpredsfiles, danetqriesfiles, danetsprtsfiles)

Before Calling Browse Images


interactive(children=(IntSlider(value=16, description='i', max=33), Output()), _dom_classes=('widget-interact'…

interactive(children=(IntSlider(value=16, description='i', max=33), Output()), _dom_classes=('widget-interact'…